In [1]:
import os
import random
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from scipy import stats
from sklearn.model_selection import KFold

from kerashypetune import KerasRandomSearch, KerasRandomSearchCV

In [2]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

np.random.seed(33)
subset_train = np.random.uniform(0,1, len(y_train))
subset_test = np.random.uniform(0,1, len(y_test))

x_train = x_train[subset_train < 0.1] / 255
y_train = y_train[subset_train < 0.1]

x_test = x_test[subset_test < 0.3] / 255
y_test = y_test[subset_test < 0.3]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((6036, 28, 28), (6036,), (2963, 28, 28), (2963,))

In [4]:
def get_model(param):
    
    set_seed(33)
        
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(param['unit_1'], activation='relu'))
    model.add(Dense(param['unit_2'], activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=param['lr']), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
param_grid = {
    'unit_1': [128,64], 
    'unit_2': stats.randint(32, 128),
    'lr': stats.loguniform(1e-4, 0.1), 
    'epochs': 100, 
    'batch_size': 512
}

# Fixed validation search

In [6]:
es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model  
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

kgs = KerasRandomSearch(hypermodel, param_grid, n_iter=15, sampling_seed=33,
                        monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(x_train, y_train, validation_data=(x_test, y_test), callbacks=[es])


15 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/15) *****
Search({'unit_1': 64, 'unit_2': 61, 'lr': 0.00026941073027491154, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00067: early stopping
SCORE: 0.93453 at epoch 62

***** (2/15) *****
Search({'unit_1': 128, 'unit_2': 105, 'lr': 0.018917299504794916, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
SCORE: 0.94836 at epoch 31

***** (3/15) *****
Search({'unit_1': 128, 'unit_2': 81, 'lr': 0.006387142993161844, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00027: early stopping
SCORE: 0.95714 at epoch 24

***** (4/15) *****
Search({'unit_1': 64, 'unit_2': 96, 'lr': 0.00016980298333942208, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00076: early stopping
SCORE: 0.93149 at epoch 7

<kerashypetune.KerasRandomSearch>

In [7]:
kgs.scores

[0.93453,
 0.94836,
 0.95714,
 0.93149,
 0.94465,
 0.95174,
 0.92575,
 0.93453,
 0.93284,
 0.9406,
 0.95174,
 0.93858,
 0.91731,
 0.94296,
 0.90786]

In [8]:
kgs.best_score

0.95714

In [9]:
kgs.best_params

{'unit_1': 128,
 'unit_2': 81,
 'lr': 0.006387142993161844,
 'epochs': 24,
 'batch_size': 512,
 'steps_per_epoch': 12}

In [10]:
kgs.best_model

# Fixed validation search Generator

In [11]:
gen = ImageDataGenerator(
         rotation_range=90,
         width_shift_range=0.1,
         height_shift_range=0.1,
         zoom_range=0.2)

In [12]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

kgs = KerasRandomSearch(hypermodel, param_grid, n_iter=15, sampling_seed=33,
                        monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(gen.flow(np.expand_dims(x_train,-1), y_train, batch_size=param_grid['batch_size'], seed=33), 
           validation_data=(np.expand_dims(x_test,-1), y_test), 
           callbacks=[es], steps_per_epoch=len(x_train)//param_grid['batch_size'])


15 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/15) *****
Search({'unit_1': 64, 'unit_2': 61, 'lr': 0.00026941073027491154, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00086: early stopping
SCORE: 0.82551 at epoch 82

***** (2/15) *****
Search({'unit_1': 128, 'unit_2': 105, 'lr': 0.018917299504794916, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00039: early stopping
SCORE: 0.86061 at epoch 34

***** (3/15) *****
Search({'unit_1': 128, 'unit_2': 81, 'lr': 0.006387142993161844, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
SCORE: 0.86736 at epoch 14

***** (4/15) *****
Search({'unit_1': 64, 'unit_2': 96, 'lr': 0.00016980298333942208, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00092: early stopping
SCORE: 0.80628 at epoch 8

<kerashypetune.KerasRandomSearch>

In [13]:
kgs.scores

[0.82551,
 0.86061,
 0.86736,
 0.80628,
 0.85116,
 0.83631,
 0.54877,
 0.75937,
 0.79447,
 0.77827,
 0.85218,
 0.84846,
 0.61323,
 0.8245,
 0.31792]

In [14]:
kgs.best_score

0.86736

In [15]:
kgs.best_params

{'unit_1': 128,
 'unit_2': 81,
 'lr': 0.006387142993161844,
 'epochs': 14,
 'batch_size': 512,
 'steps_per_epoch': 11}

In [16]:
kgs.best_model

# Cross validation search

In [17]:
X = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])

X.shape, y.shape

((8999, 28, 28), (8999,))

In [18]:
cv = KFold(n_splits=3, random_state=33, shuffle=True)

es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)

kgs = KerasRandomSearchCV(hypermodel, param_grid, cv=cv, n_iter=5, sampling_seed=33,
                          monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(X, y, callbacks=[es])


##################
###  Fold 001  ###
##################

5 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/5) *****
Search({'unit_1': 64, 'unit_2': 97, 'lr': 0.0005535560552210636, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
SCORE: 0.92667 at epoch 36

***** (2/5) *****
Search({'unit_1': 128, 'unit_2': 77, 'lr': 0.0025150330161023593, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00035: early stopping
SCORE: 0.942 at epoch 25

***** (3/5) *****
Search({'unit_1': 128, 'unit_2': 37, 'lr': 0.0005324197618194066, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Epoch 00037: early stopping
SCORE: 0.93733 at epoch 27

***** (4/5) *****
Search({'unit_1': 64, 'unit_2': 114, 'lr': 0.0005304760422961851, 'epochs': 100, 'batch_size': 512})
Restoring model weights from the end of the best epoch.
Ep

<kerashypetune.KerasRandomSearchCV>

In [19]:
kgs.folds_scores

{'fold 1': [0.92667, 0.942, 0.93733, 0.93533, 0.951],
 'fold 2': [0.93767, 0.95367, 0.94333, 0.93867, 0.95533],
 'fold 3': [0.93765, 0.95365, 0.94165, 0.94532, 0.95499]}

In [20]:
kgs.folds_best_score

{'fold 1': 0.951, 'fold 2': 0.95533, 'fold 3': 0.95499}

In [21]:
kgs.folds_best_params

{'fold 1': {'unit_1': 128,
  'unit_2': 125,
  'lr': 0.010310596407937588,
  'epochs': 18,
  'batch_size': 512,
  'steps_per_epoch': 12},
 'fold 2': {'unit_1': 128,
  'unit_2': 125,
  'lr': 0.010310596407937588,
  'epochs': 12,
  'batch_size': 512,
  'steps_per_epoch': 12},
 'fold 3': {'unit_1': 128,
  'unit_2': 125,
  'lr': 0.010310596407937588,
  'epochs': 24,
  'batch_size': 512,
  'steps_per_epoch': 12}}

In [22]:
kgs.folds_best_models

{'fold 1': <tensorflow.python.keras.engine.sequential.Sequential at 0x17fc7b1d0f0>,
 'fold 2': <tensorflow.python.keras.engine.sequential.Sequential at 0x17fc92cca20>,
 'fold 3': <tensorflow.python.keras.engine.sequential.Sequential at 0x17fc93285f8>}

In [23]:
kgs.best_params_score

0.95377

In [24]:
kgs.best_params

[{'unit_1': 128,
  'unit_2': 125,
  'lr': 0.010310596407937588,
  'epochs': 18,
  'batch_size': 512,
  'steps_per_epoch': 12},
 {'unit_1': 128,
  'unit_2': 125,
  'lr': 0.010310596407937588,
  'epochs': 12,
  'batch_size': 512,
  'steps_per_epoch': 12},
 {'unit_1': 128,
  'unit_2': 125,
  'lr': 0.010310596407937588,
  'epochs': 24,
  'batch_size': 512,
  'steps_per_epoch': 12}]